In [1]:
import PyQt5
%matplotlib qt
import math
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
print("setup done")

setup done


In [77]:
# df = pd.read_csv('./dataset/brazil_report.csv', sep=';', parse_dates=['Sale Date Time'])
# del df['Employee']
# df['Date'] = pd.to_datetime(df['Sale Date Time'])
# df = df.drop(columns=['Sale Date Time'])
# df = df.loc[(df['Client'] != 'Customer not informed')]
df = pd.read_pickle("./data_rating_init.pkl")
df = df.loc[(df['Client'] != 'Customer not informed')]
print("dataset Loaded")

dataset Loaded


In [72]:
df.head()

,Company Code,Order Number,Product,Product Category,Client,Client City,Product Cost,Discount Amount,Amount,Total,Form of payment,Date,ProductId,Rating_rf_combined_fin
77,39000,234,Special Gasoline,Fuel,Vickie James,Agririe,3.05,0.0,28.177,100.00,Debit Card,2017-03-31 08:09:00,06945203018,4.9
104,39000,308,Special Gasoline,Fuel,Bennie Chavez,Agririe,3.05,0.0,28.177,100.00,Money,2017-03-31 08:56:00,06945203018,4.9
165,39000,447,Special Gasoline,Fuel,Lois Anderson,Agririe,3.05,0.0,34.661,123.01,Credit Card,2017-03-31 10:26:00,06945203018,4.9
193,39000,527,Special Gasoline,Fuel,Melanie Ortega,Devale,3.05,0.0,32.041,113.71,Money,2017-03-31 11:34:00,06945203018,4.9
248,39000,1478,Special Gasoline,Fuel,Toby Watts,Iwreridge,3.05,0.0,36.909,130.99,Debit Card,2017-03-31 13:00:00,06945203018,4.9


In [78]:
df_bk = df.copy()

In [126]:
df = df_bk.copy()

In [98]:
tmp = df.loc[df['Client']=='Bennie Chavez'].groupby('ProductId')['Order Number'].count().reset_index()
tmp.columns=['ProductId','IndiRating']
tmp['Client']='Bennie Chavez'

In [101]:
tmp = df.loc[df['Client']=='Vickie James'].groupby('ProductId')['Order Number'].count().reset_index()
tmp.columns=['ProductId','IndiRating']
tmp['Client']='Vickie James'

In [102]:
df = pd.merge(df, tmp, on=['Client','ProductId'],how='left')

In [88]:
df = pd.merge(df, tmp, on=['Client','ProductId'],how='left')

In [107]:
df.head()

,Company Code,Order Number,Product,Product Category,Client,Client City,Product Cost,Discount Amount,Amount,Total,Form of payment,Date,ProductId,Rating_rf_combined_fin,IndiRating_x,IndiRating_y,cc
0,39000,234,Special Gasoline,Fuel,Vickie James,Agririe,3.05,0.0,28.177,100.00,Debit Card,2017-03-31 08:09:00,06945203018,4.9,NaN,1.0,0
1,39000,308,Special Gasoline,Fuel,Bennie Chavez,Agririe,3.05,0.0,28.177,100.00,Money,2017-03-31 08:56:00,06945203018,4.9,1.0,NaN,0
2,39000,447,Special Gasoline,Fuel,Lois Anderson,Agririe,3.05,0.0,34.661,123.01,Credit Card,2017-03-31 10:26:00,06945203018,4.9,NaN,NaN,0
3,39000,527,Special Gasoline,Fuel,Melanie Ortega,Devale,3.05,0.0,32.041,113.71,Money,2017-03-31 11:34:00,06945203018,4.9,NaN,NaN,0
4,39000,1478,Special Gasoline,Fuel,Toby Watts,Iwreridge,3.05,0.0,36.909,130.99,Debit Card,2017-03-31 13:00:00,06945203018,4.9,NaN,NaN,0


In [109]:
# df.drop(['as'], axis=1)

In [104]:
df.head()

,Company Code,Order Number,Product,Product Category,Client,Client City,Product Cost,Discount Amount,Amount,Total,Form of payment,Date,ProductId,Rating_rf_combined_fin,IndiRating_x,IndiRating_y
0,39000,234,Special Gasoline,Fuel,Vickie James,Agririe,3.05,0.0,28.177,100.00,Debit Card,2017-03-31 08:09:00,06945203018,4.9,NaN,1.0
1,39000,308,Special Gasoline,Fuel,Bennie Chavez,Agririe,3.05,0.0,28.177,100.00,Money,2017-03-31 08:56:00,06945203018,4.9,1.0,NaN
2,39000,447,Special Gasoline,Fuel,Lois Anderson,Agririe,3.05,0.0,34.661,123.01,Credit Card,2017-03-31 10:26:00,06945203018,4.9,NaN,NaN
3,39000,527,Special Gasoline,Fuel,Melanie Ortega,Devale,3.05,0.0,32.041,113.71,Money,2017-03-31 11:34:00,06945203018,4.9,NaN,NaN
4,39000,1478,Special Gasoline,Fuel,Toby Watts,Iwreridge,3.05,0.0,36.909,130.99,Debit Card,2017-03-31 13:00:00,06945203018,4.9,NaN,NaN


In [127]:
client_list = list(df['Client'].unique())
df['FinRating_indi'] = 0
for client in client_list:
#     df_client = df.loc[df['Client']==client]
    tmp = df.loc[df['Client']==client].groupby('ProductId')['Order Number'].count().reset_index()
    tmp.columns=['ProductId','IndiRating']
    tmp['Client']= client
    df = pd.merge(df, tmp, on=['Client','ProductId'],how='left')
    df['IndiRating'].fillna(0, inplace=True)
    df['FinRating_indi']= df['FinRating_indi']+ df['IndiRating']
    df = df.drop(['IndiRating'], axis=1)

KeyboardInterrupt: 

In [8]:
# # Frequency
# dffreq = df.groupby([['Client','ProductId']])
# # dffreq.columns = ['Client','Frequency']
# # dfclient = pd.merge(dfclient, dffreq, on='Client')

ValueError: Grouper and axis must be same length